# LangChain Experiement Number 1

In [42]:
import openai
import os
import dotenv
dotenv.load_dotenv()
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

## Sample Project

In [43]:
project_details="""
Basically you would put your hackathon idea in a text box
and we would run 3 llm chains in the background
One for frontend one for backend, and then feed both chains into the last one which evaluates the two first chains if its feasible or not, and if it isn't, calls the first two chains again
The first two will also return design specifications, eg an outline of the frontend and backend
maybe another textbox in the frontend for skills/target categories
"""
project_technologies="""
OpenAI API
StreamLit
"""

### Setting Up Chain No. 1: Feature Retrieval

In [44]:
llm = ChatOpenAI(temperature=0, max_tokens=1000, model="gpt-3.5-turbo")
prompt = PromptTemplate(
    input_variables=["project_details", "project_technologies"],
    template="""
    Given the following project description and tech stack, identify and elaborate on the key frontend features that would be necessary for development. The frontend features should not involve backend api calls or database interactions. The frontend features should be described in terms of user stories or detailed feature requirements.
    This project is a hackathon project. Break apart the features into MVP and additional features. The MVP should be the minimum features necessary to have a working prototype.
    Project description: {project_details}
    Technologies: {project_technologies}
    """
)

chain = LLMChain(llm=llm, prompt=prompt)
frontend_features = chain.run({
    'project_details': project_details,
    'project_technologies': project_technologies
})
print(frontend_features)

MVP Frontend Features:
1. User Registration: Allow users to create an account using their email address and password. This feature will enable users to save their hackathon ideas and track their progress.
2. Login: Provide a login functionality for registered users to access their accounts and previously saved hackathon ideas.
3. Hackathon Idea Submission: Implement a text box where users can input their hackathon ideas. This feature will allow users to submit their ideas for evaluation.
4. Design Specification Display: Display the design specifications generated by the llm chain for both the frontend and backend. This feature will provide users with an outline of the proposed design for their hackathon project.
5. Feasibility Evaluation: Show the evaluation result of the last llm chain, indicating whether the proposed hackathon idea is feasible or not. If it is not feasible, prompt the user to modify their idea and resubmit.
6. Skills/Target Categories Selection: Include a textbox or 

## Setting Up Chain Two:  Feature Extraction

In [45]:
specification_prompt = PromptTemplate(
    input_variables=["frontend_features", "project_technologies"],
    template="""
    Given the extracted frontend features {frontend_features} and the specified skills/technologies {project_technologies}, create a detailed technical specification. 
    This specification should include the technologies to be used, the architecture, pages to be developed, and the components required for each page.
    You should ignore the technologies for the backend and focus on the frontend.
    """
)

specification_chain = LLMChain(llm=llm, prompt=specification_prompt)
specification = specification_chain.run({
    'frontend_features': frontend_features,
    'project_technologies': project_technologies
})
print(specification)

Technical Specification:

Technologies:
1. OpenAI API: To integrate natural language processing capabilities for user registration, login, hackathon idea submission, and feasibility evaluation.
2. Streamlit: To develop the frontend application and create interactive web pages.
3. HTML/CSS: To design and style the web pages.
4. JavaScript: To add interactivity and dynamic behavior to the web pages.
5. Git: To manage version control and collaboration during development.

Architecture:
The frontend application will follow a client-server architecture, where the Streamlit application will act as the client and interact with the OpenAI API for natural language processing tasks. The application will be developed using the Model-View-Controller (MVC) design pattern.

Pages to be Developed:
1. User Registration Page:
   - Components: 
     - Email input field
     - Password input field
     - Register button
   - Functionality:
     - Validate email and password inputs
     - Send registratio

## Setting Up Chain Three: AI Approval

In [59]:
approval_prompt = PromptTemplate(
    input_variables=["technical_specification", "aspect", "group_size"],
    template="""
    Given the developed technical specification {technical_specification}, conduct a thorough review for any inconsistencies or issues. 
    This specification is only for the {aspect} aspect of the project, and should not be evaluated for other aspects, such as backend and database requirements.
    Evaluate whether the project, as described, can be realistically completed within the 2 day hackathon for {group_size} people.
    If it meets all the criteria, output '1'. If not, output '0'. In the next line, provide comments explaining the reasons for disapproval, if any.
    """
)

approval_chain = LLMChain(llm=llm, prompt=approval_prompt)
approval = approval_chain.run({
    'technical_specification': specification,
    'aspect': 'frontend',
    'group_size': '4'
})

print(approval)

0
The project, as described, cannot be realistically completed as a proof of concept within the 2-day hackathon for 4 people. The scope of the project is quite extensive, with multiple pages and functionalities to be developed. Additionally, integrating the OpenAI API for natural language processing tasks may require additional time and effort. It is recommended to prioritize and simplify the project scope to ensure it can be completed within the given timeframe.


## Setting Up Chain Four: AI Refinement

In [58]:
refinement_prompt = PromptTemplate(
    input_variables=["project_details", "frontend_features", "project_technologies", "technical_specification", "aspect", "group_size", "approval"],
    template="""
    Refine the project based on the following information:
    Project Description: {project_details}
    Frontend Features: {frontend_features}
    Project Technologies: {project_technologies}
    Technical Specification: {technical_specification}
    Aspect: {aspect}
    Group Size: {group_size}
    Approval Status: {approval}
    If approval is '0', update frontend features and technical specification to meet the criteria, ignoring backend and database.
    """
)

refinement_chain = LLMChain(llm=llm, prompt=refinement_prompt)
refinement = refinement_chain.run({
    'project_details': project_details,
    'frontend_features': frontend_features,
    'project_technologies': project_technologies,
    'technical_specification': specification,
    'aspect': 'frontend',
    'group_size': '4',
    'approval': approval
})
print(refinement)


Frontend Features: MVP Frontend Features:
1. User Registration: Allow users to create an account using their email address and password. This feature will enable users to save their hackathon ideas and track their progress.
2. Login: Provide a login functionality for registered users to access their accounts and previously saved hackathon ideas.
3. Hackathon Idea Submission: Implement a text box where users can input their hackathon ideas. This feature will allow users to submit their ideas for evaluation.
4. Design Specification Display: Display the design specifications generated by the llm chain for both the frontend and backend. This feature will provide users with an outline of the proposed design for their hackathon project.
5. Feasibility Evaluation: Show the evaluation result of the last llm chain, indicating whether the proposed hackathon idea is feasible or not. If it is not feasible, prompt the user to modify their idea and resubmit.
6. Skills/Target Categories Selection: In